In [17]:
%matplotlib qt
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import mne

In [ ]:
fname = Path.cwd().parent
fname = "/Users/payamsadeghishabestari/antinomics_clean_codes/subjects/dvob/EEG/regularity/dvob_regularity.vhdr"
raw = mne.io.read_raw_brainvision(fname, preload=True)

Extracting parameters from /Users/payamsadeghishabestari/antinomics_clean_codes/subjects/dvob/EEG/regularity/dvob_regularity.vhdr...
Setting channel info structure...
Reading 0 ... 2210759  =      0.000 ...  2210.759 secs...


/var/folders/20/hsy69tx529ndn3rkv5gzcf0c0000gn/T/ipykernel_3386/2842641597.py:2: RuntimeWarning: No coordinate information found for channels ['X', 'Y', 'Z', 'Photo', 'Audio']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(fname, preload=True)
/var/folders/20/hsy69tx529ndn3rkv5gzcf0c0000gn/T/ipykernel_3386/2842641597.py:2: RuntimeWarning: Not setting positions of 8 misc channels found in montage:
['Pulse', 'GSR', 'X', 'Y', 'Z', 'Resp', 'Photo', 'Audio']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(fname, preload=True)


In [ ]:
## read data
subject_id = "dvob"
fname = Path.cwd().parent.parent.parent / "subjects" / subject_id / "EEG" / "regularity" / f"{subject_id}_regularity.vhdr"
raw = mne.io.read_raw_brainvision(fname, preload=True)
raw.resample(250)

## get events and create blocks
diff_thr = 750
events, events_dict = mne.events_from_annotations(raw)
events_orig = events.copy() # better to keep the original
ev_ids = [i for i in range(1, 19) if i not in (9, 10)]
events = events[np.isin(events[:, -1], ev_ids)]
split_indices = np.where(np.diff(events[:, 0]) > diff_thr)[0] + 1
blocks = np.split(events, split_indices)

## check balancing
for block_idx, block in enumerate(blocks):
    assert len(block) == 500, f"Number of triggers in block {block_idx + 1} is {len(block)}, must be 500." 


## concatenating similar blocks
std_ord_block_idxs = [0, 2, 4]
std_rnd_block_idxs = [1, 3, 5]
tin_ord_block_idxs = [6, 8, 10]
tin_rnd_block_idxs = [7, 9, 11]

first_event = [0, 0, 99999] 
blocks_dict = {}
for block_idxs, title in zip([std_ord_block_idxs, std_rnd_block_idxs, \
                                tin_ord_block_idxs, tin_rnd_block_idxs], \
                                ["std_ord", "std_rnd", "tin_ord", "tin_rnd"]):  
    
    blocks_dict[title] = np.concatenate(
                                        [blocks[i] for i in block_idxs]
                                        )
    blocks_dict[title] = np.insert(blocks_dict[title], 0, first_event, axis=0)
    


## entropy level decoding
## 4 targets std_or, std_rnd, tin_or, tin_rnd
## so here all epochs X, y (not important which frequency they are, finally y should be shape (,4))


## sound-to-sound decoding
## 4 targets, no matter which block... (y should be again (,4))


## binary classification for 2 groups


## some plots

## source



## 4 targets, f1, f2, f3, f4 of standard -> clf_std trained on rnd ones (blocks 1, 3, 5)
## 4 targets, f1, f2, f3, f4 of tinnitus -> clf_tin trained on rnd ones (blocks 7, 9, 11)



Extracting parameters from /Users/payamsadeghishabestari/antinomics_clean_codes/subjects/dvob/EEG/regularity/dvob_regularity.vhdr...
Setting channel info structure...
Reading 0 ... 2210759  =      0.000 ...  2210.759 secs...


/var/folders/20/hsy69tx529ndn3rkv5gzcf0c0000gn/T/ipykernel_3386/2626081921.py:3: RuntimeWarning: No coordinate information found for channels ['X', 'Y', 'Z', 'Photo', 'Audio']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(fname, preload=True)
/var/folders/20/hsy69tx529ndn3rkv5gzcf0c0000gn/T/ipykernel_3386/2626081921.py:3: RuntimeWarning: Not setting positions of 8 misc channels found in montage:
['Pulse', 'GSR', 'X', 'Y', 'Z', 'Resp', 'Photo', 'Audio']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(fname, preload=True)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S  4', 'Stimulus/S  5', 'Stimulus/S  6', 'Stimulus/S  7', 'Stimulus/S  8', 'Stimulus/S 11', 'Stimulus/S 12', 'Stimulus/S 13', 'Stimulus/S 14', 'Stimulus/S 15', 'Stimulus/S 16', 'Stimulus/S 17', 'Stimulus/S 18', 'Stimulus/S140']


In [ ]:
ev_id = 8

indices_of_carrier = np.where(blocks[1][:, -1] == ev_id)[0]
selected_events = blocks[1][indices_of_carrier - 1]
unique, counts = np.unique(selected_events[:, 2], return_counts=True)
print(unique)
print(counts)
counts_dict = dict(zip(unique, counts))
min_count = min(counts_dict.values()) # the minimum count to balance

# Randomly downsample each trial type to the minimum count
balanced_indices = []
for trial_id in unique:
    trial_indices = np.where(selected_events == ev_id)[0]
    selected = np.random.choice(trial_indices, min_count, replace=False)
    balanced_indices.extend(indices_of_carrier[selected - 1])

len(blocks[1][balanced_indices])

[5 6 7 8]
[39 32 25 30]


100

In [127]:
sample_events = blocks_dict["std_rnd"]

In [128]:
sample_events

array([[     0,      0,  99999],
       [ 56350,      0,      8],
       [ 56434,      0,      7],
       ...,
       [277285,      0,      7],
       [277368,      0,      5],
       [277452,      0,      8]])

In [ ]:
ev_id = 7

indices_of_carrier = np.where(sample_events[:, -1] == ev_id)[0]
selected_events = sample_events[indices_of_carrier - 1]
unique, counts = np.unique(selected_events[:, 2], return_counts=True)
print(unique, counts)
min_count = min(counts) # the minimum count to balance

# find min
balanced_indices = []
for trial_id in unique:
    trial_indices = np.where(selected_events == trial_id)[0]
    selected = np.random.choice(trial_indices, min_count, replace=False)
    balanced_indices.append(indices_of_carrier[selected - 1])

# equalize from beginning
len(sample_events[balanced_indices[1]])

[5 6 7 8] [79 83 94 93]


79

In [ ]:
## copied from sequence generation code

'''
experiment_blocks = [
    ("Standard Orderly 1", standard_orderly_blocks[0]),
    ("Standard Random 1", standard_random_blocks[0]),
    ("Standard Orderly 2", standard_orderly_blocks[1]),
    ("Standard Random 2", standard_random_blocks[1]),
    ("Standard Orderly 3", standard_orderly_blocks[2]),
    ("Standard Random 3", standard_random_blocks[2]),
    ("Custom Orderly 1", custom_orderly_blocks[0]),
    ("Custom Random 1", custom_random_blocks[0]),
    ("Custom Orderly 2", custom_orderly_blocks[1]),
    ("Custom Random 2", custom_random_blocks[1]),
    ("Custom Orderly 3", custom_orderly_blocks[2]),
    ("Custom Random 3", custom_random_blocks[2])
]


# Mapping of names to triggers
trigger_dict = {
    "f1_std_or": 1, "f2_std_or": 2, "f3_std_or": 3, "f4_std_or": 4,
    "f1_std_rndm": 5, "f2_std_rndm": 6, "f3_std_rndm": 7, "f4_std_rndm": 8,
    "f1_tin_or": 11, "f2_tin_or": 12, "f3_tin_or": 13, "f4_tin_or": 14,
    "f1_tin_rndm": 15, "f2_tin_rndm": 16, "f3_tin_rndm": 17, "f4_tin_rndm": 18
}
'''